# Efficient Yelp API Calls (Core)
Nena Esaw

Instructions:

For this assignment, you will be working with the Yelp API.

As before, you will use the Yelp API to search your favorite city for a cuisine type of your choice.

Extract all of the results from your search and compile them into one dataframe using a for loop as shown in the lesson "Code for Efficient API Extraction"

In [42]:
import json 
from yelpapi import YelpAPI
import pandas as pd 
from tqdm.notebook import tqdm_notebook
import time, math
import os 


#### Login 

In [6]:
# Load API Credentials
with open("C:\\Users\\nena_\\.secret\\yelp_api.json") as f: 
    login = json.load(f)
# Instantiate YelpAPI Variable
yelp_api = YelpAPI(login['api-key'], timeout_s=5.0)



#### Define Search 

In [8]:
#set our API call parameters 
LOCATION = 'Kansas City, KS'
TERM = 'Burger'

In [46]:
#create a results-in-progess JSON file 
JSON_FILE = "Data/results_in_progress_KS_burger.json"
JSON_FILE 

'Data/results_in_progress_KS_burger.json'

#### Check for JSON File and create results 


In [47]:
def create_json_file(JSON_FILE, delete_if_exists=False):
    # Check if JSON_FILE exists
    file_exists = os.path.isfile(JSON_FILE)
    
    # If it DOES exist:
    if file_exists:
        # Check if user wants to delete if exists
        if delete_if_exists:
            print(f"[!] {JSON_FILE} already exists. Deleting previous file...")
            # Delete file and confirm it no longer exists
            os.remove(JSON_FILE)
            # Recursive call to function after old file deleted
            create_json_file(JSON_FILE, delete_if_exists=False)
        else:
            print(f"[i] {JSON_FILE} already exists.")
    # If it does NOT exist:
    else:
        # INFORM USER AND SAVE EMPTY LIST
        print(f"[i] {JSON_FILE} not found. Saving empty list to new file.")
        
        # CREATE ANY NEEDED FOLDERS
        # Get the Folder Name only
        folder = os.path.dirname(JSON_FILE)
        
        # If JSON_FILE includes a folder:
        if len(folder) > 0:
            # Create the folder
            os.makedirs(folder, exist_ok=True)
        
        # Save empty list to start the json file
        with open(JSON_FILE, 'w') as f:
            json.dump([], f)

In [43]:
## Create a new empty json file (exist the previous if it exists)
create_json_file(JSON_FILE, delete_if_exists=True)
## Load previous results and use len of results for offset
with open(JSON_FILE,'r') as f:
    previous_results = json.load(f)
    
## set offset based on previous results
n_results = len(previous_results)
print(f'- {n_results} previous results found.')
# use our yelp_api variable's search_query method to perform our API call
results = yelp_api.search_query(location=LOCATION,
                                term=TERM,
                               offset=n_results)
## How many results total?
total_results = results['total']
## How many did we get the details for?
results_per_page = len(results['businesses'])
# Use math.ceil to round up for the total number of pages of results.
n_pages = math.ceil((results['total']-n_results)/ results_per_page)
n_pages



[!] Data/results_in_progress_KS_burger.json already exists. Deleting previous file...
[i] Data/results_in_progress_KS_burger.json not found. Saving empty list to new file.
- 0 previous results found.


75

In [48]:
for i in tqdm_notebook( range(1,n_pages+1)):
    
    ## Read in results in progress file and check the length
    with open(JSON_FILE, 'r') as f:
        previous_results = json.load(f)
    ## save number of results for to use as offset
    n_results = len(previous_results)
    
    if (n_results + results_per_page) > 1000:
        print('Exceeded 1000 api calls. Stopping loop.')
        break
    
    ## use n_results as the OFFSET 
    results = yelp_api.search_query(location=LOCATION,
                                    term=TERM, 
                                    offset=n_results)
    
    
    
    ## append new results and save to file
    previous_results.extend(results['businesses'])
    
    # display(previous_results)
    with open(JSON_FILE,'w') as f:
        json.dump(previous_results,f)
    
    time.sleep(.2)



  0%|          | 0/75 [00:00<?, ?it/s]

Exceeded 1000 api calls. Stopping loop.


#### Load final results 

In [49]:
# load final results
final_df = pd.read_json(JSON_FILE)
display(final_df.head(), final_df.tail())

,id,alias,name,image_url,is_closed,url,review_count,categories,rating,coordinates,transactions,price,location,phone,display_phone,distance
0,cC6V4pEnT_-4C61OZZOWug,mason-jar-kansas-city-2,Mason Jar,https://s3-media3.fl.yelpcdn.com/bphoto/PX_XYe...,False,https://www.yelp.com/biz/mason-jar-kansas-city...,364,"[{'alias': 'bars', 'title': 'Bars'}, {'alias':...",4.0,"{'latitude': 39.1153106090531, 'longitude': -9...",[delivery],$$,"{'address1': '941 N 74th Dr', 'address2': '', ...",+19132428165,(913) 242-8165,2830.097258
1,MMMYsxyTDI25N0sNlA72cg,tays-burger-shack-north-kansas-city,Tay's Burger Shack,https://s3-media2.fl.yelpcdn.com/bphoto/r1X46e...,False,https://www.yelp.com/biz/tays-burger-shack-nor...,489,"[{'alias': 'burgers', 'title': 'Burgers'}]",4.5,"{'latitude': 39.1433722382707, 'longitude': -9...",[delivery],$$,"{'address1': '1019 Armour Rd', 'address2': '',...",+18165418282,(816) 541-8282,15048.023395
2,gEY_gKla4AxWh9bKGEYqjg,snackle-box-kansas-city,Snackle Box,https://s3-media3.fl.yelpcdn.com/bphoto/0UnSf6...,False,https://www.yelp.com/biz/snackle-box-kansas-ci...,25,"[{'alias': 'tradamerican', 'title': 'American ...",4.5,"{'latitude': 39.1644724869275, 'longitude': -9...",[delivery],NaN,"{'address1': '4208 West Dr', 'address2': '', '...",+19132619730,(913) 261-9730,9187.373942
3,c35QD0-OaTg-9ADx9rgnUg,the-lunch-box-kansas-city-5,The Lunch Box,https://s3-media2.fl.yelpcdn.com/bphoto/vSyeW4...,False,https://www.yelp.com/biz/the-lunch-box-kansas-...,144,"[{'alias': 'burgers', 'title': 'Burgers'}, {'a...",4.5,"{'latitude': 39.10415, 'longitude': -94.60614}","[delivery, pickup]",$,"{'address1': '1701 W 9th St', 'address2': '', ...",+18164747526,(816) 474-7526,10822.850845
4,FFRhPjhtDmirTUYJjJBBkg,brgr-kitchen-bar-kansas-city,BRGR kitchen + bar,https://s3-media1.fl.yelpcdn.com/bphoto/XxwxCw...,False,https://www.yelp.com/biz/brgr-kitchen-bar-kans...,650,"[{'alias': 'burgers', 'title': 'Burgers'}, {'a...",4.0,"{'latitude': 39.0970137, 'longitude': -94.5830...","[delivery, pickup]",$$,"{'address1': '11 E 14th St', 'address2': '', '...",+18162212747,(816) 221-2747,12772.181780


,id,alias,name,image_url,is_closed,url,review_count,categories,rating,coordinates,transactions,price,location,phone,display_phone,distance
995,nkkWA7Xbdz6RyAphxz2qAw,new-day-cafe-olathe-2,New Day Cafe,https://s3-media1.fl.yelpcdn.com/bphoto/OQCkXa...,False,https://www.yelp.com/biz/new-day-cafe-olathe-2...,3,"[{'alias': 'breakfast_brunch', 'title': 'Break...",3.5,"{'latitude': 38.897247, 'longitude': -94.778474}",[],NaN,"{'address1': '12755 S Mur-Len Rd', 'address2':...",+19133134940,(913) 313-4940,22343.936259
996,xxfh59jPMh5ah1loTEnKFg,sama-zama-kansas-city,Sama Zama,https://s3-media1.fl.yelpcdn.com/bphoto/Oxtn2q...,False,https://www.yelp.com/biz/sama-zama-kansas-city...,269,"[{'alias': 'japanese', 'title': 'Japanese'}]",3.5,"{'latitude': 39.0563707, 'longitude': -94.589731}","[pickup, delivery]",$$,"{'address1': '3906 Central St', 'address2': ''...",+18167563600,(816) 756-3600,12917.558381
997,Ei_zL68qFe-EqV_PG_HFPw,wendys-independence-5,Wendy's,https://s3-media1.fl.yelpcdn.com/bphoto/K03vLO...,False,https://www.yelp.com/biz/wendys-independence-5...,22,"[{'alias': 'hotdogs', 'title': 'Fast Food'}, {...",1.5,"{'latitude': 39.042623, 'longitude': -94.414924}",[delivery],$,"{'address1': '4301 S. Noland Rd.', 'address2':...",+18163735785,(816) 373-5785,27897.123809
998,WGlveOnqJckbqtZRBExRpA,tanners-bar-and-grill-kansas-city-10,Tanner's Bar & Grill,https://s3-media4.fl.yelpcdn.com/bphoto/vh_6Nv...,False,https://www.yelp.com/biz/tanners-bar-and-grill...,73,"[{'alias': 'sportsbars', 'title': 'Sports Bars...",3.0,"{'latitude': 39.057125, 'longitude': -94.6119449}",[delivery],$$,"{'address1': '3900 Rainbow Blvd', 'address2': ...",+19132837367,(913) 283-7367,11074.439315
999,BM7wykdhR_V742UDF9dBjA,old-chicago-kansas-city-4,Old Chicago,https://s3-media3.fl.yelpcdn.com/bphoto/eJVDlM...,False,https://www.yelp.com/biz/old-chicago-kansas-ci...,146,"[{'alias': 'pizza', 'title': 'Pizza'}, {'alias...",2.5,"{'latitude': 39.24322, 'longitude': -94.47664}","[pickup, delivery]",$$,"{'address1': '8251 N Booth Ave', 'address2': '...",+18167818336,(816) 781-8336,27449.168252
